# Finding Recurring Payments in Bank Statements

## Info

- See final cell for complete solution
- To install all required packages to your environment:
`$ pip install -r requirements.txt`

## Prior Reading

- Financial Forecasting and Analysis for Low-Wage Workers: https://arxiv.org/pdf/1806.05362.pdf
    - Good information on extracting recurring transactions and forecasting balance
    - However, extraction works on defined recurrence periods (monthly, semi-monthly, weekly, bi-weekly)
    - I want the extraction to be period-agnostic, but still be able to return the period in days
    
    
## Rough strategy
I see the problem falling roughly into 2 stages:
- Clustering transactions by payee/payer
- Classifying clusters as recurring or non-recurring

For clustering I only used the transaction description

For classifying I used the number of days between payments

This means I did not use the amount feature. I felt it wasn't suitable to be used in the clustering because I don't think we want a change in payment amount to indicate a different series of payments. For example with Spotify, if you upgraded to the family plan for £14.99, I wouldn't want those transactions to be in a different cluster than previous payments of £9.99. I also didn't use it for classifying payments as recurring, due to amount not being mentioned in the definition in the problem statement.
> The definition of a regularly recurring transaction is any debit or credit transaction to or from the same entity that happens at regular intervals.


## Data import and exploration

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('example_data.csv')

df.info()

1210 rows, no NULL values at all (good!)

First observation is that the made_on column is an object, not a datetime; fixing this will make our lives easier later

In [ ]:
#Convert made_on column to datetime
df['made_on'] = pd.to_datetime(df['made_on'])

Inspection shows first column is just an index, and `user_id` is OpenBanking user_id. Neither are useful so will drop both:

In [ ]:
#drop first two columns of df
df = df.drop(df.columns[:2], axis = 1)

In [ ]:
#separate df into credits and debits
credit_df = df[df['income'] == True]
debit_df = df[df['income'] == False]

print(f'Number of credits = {len(credit_df)}')
print(f'Number of debits = {len(debit_df)}')

Big imbalance between the amount of credits and debits; in another case this may indicate using a different method for credits and debits could be useful.

Going to make a slightly clearer column for later:

In [ ]:
df['type'] = df['income'].map(lambda x: 'credit' if x == True else 'debit')

df[['type', 'income']].head()

## Description cleaning

Generally most of the performance when clustering text data will come from proper preprocessing.

The typical transaction string contains useful information, but is messy.

E.G. `TFL TRAVEL CH ON 03 APR CLP` Tells us that the transaction was for TFL Travel on the 3rd April and that the transaction was a Charge (rather than a Refund)

Hopefully removing these common strings can help us cluster payments better, since we do not want to cluster all Direct Debits or Fund Transfers together.

`DDR` or `D/D` also appears in transactions, meaning direct debit. This may prove useful as a feature in future, however was not necessary in the method I ended up with, so I removed those codes too.

In [ ]:
#Big Regex pattern to remove undesirable parts of the description
# \b begins and ends the pattern
# | indicates OR
# \d{2} - 2-digit numbers - commonly used for day in date
# JAN-DEC - commonly used for month in date
# ON|BCC|CLP|ATM|CH|FT|BGC - common abbreviations that indicate the type of transaction

date_pattern = r'\b(\d{2}|JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC|ON|BCC|CLP|ATM|CH|FT|BGC|DDR|\')\b'
df['clean_desc'] = df['original_description'].str.replace(date_pattern, '')

#replace '*', '.', or ',' with a space
df['clean_desc'] = df['clean_desc'].str.replace(r'\*|\.|\,',' ', regex = True)

#convert to lower case
df['clean_desc'] = df['clean_desc'].str.lower()

There are still some dates and messy bits in there E.G.:

In [ ]:
df.loc[31,'clean_desc']

but working on an 80-20 basis this feels good enough. Descriptions are now much cleaner:

In [ ]:
df[['original_description', 'clean_desc']].head(10)

From here, I had a go at vectorising the description string using TF-IDF, and clustering with K-Means. However, 3 issues came with this:
- Sample size isn't really big enough for good accuracy
- No ideal value for K without manually checking the transactions, which is ungeneralisable
- I don't really have enough time to model this properly!

Instead I ended up doing something quite primitive, but effective; taking the first word of each transaction's cleaned description and combining with either credit or debit type (to make sure we don't combine recurring credits and debits to the same payer/payee).

In [ ]:
#take just the first word (splitting by space) and concatenate with a space and credit/debit
df['keyword'] = df['clean_desc'].map(lambda x: x.split(' ', 1)[0]) + ' ' + df['type']

In [ ]:
len(df['keyword'].unique())

This leaves us with 278 unique transaction descriptions to start classifying as recurring or non-recurring.

This isn't completely fool-proof however! There are 4 transactions where the original description are websites:

In [ ]:
df[df['keyword'] == 'www debit']

These have been grouped into 1 `www debit` group, which isn't ideal, but, again, 80-20.

(On retrospect, the PureGym one is the first payment in a recurring series, it would have been nice to capture this one too.)

## Finding recurring payments

We're generally going to find recurring payments by looking at the period between payments.

In order to do that I first need to calculate the period for each cluster of transactions:

In [ ]:
def daydelta(df, date_col):
    #Param df: dataframe containing the date column
    #Param date_col: string of the name of the date column to be delta'd
    #Returns dataframe with daydelta column appended
    #Sort dataframe by date column
    process_df = df.sort_values(date_col)
    #Make new column that is the date_col minus the date_column in the previous row
    process_df['daydelta'] = process_df[date_col] - process_df[date_col].shift()
    return process_df

def get_subdf(df, keyword):
    #Small function to find just the rows containing each keyword, ordered by date
    #Param df: dataframe to be searched
    #Param keyword: keyword to be looked for
    sub_df = df[df['keyword'] == keyword]
    #Sort values by date
    sub_df = sub_df.sort_values('made_on')
    return sub_df

#Add the daydelta column to the dataframe containing keyword 'spotify debit'
daydelta(get_subdf(df, 'spotify debit'), 'made_on')

Now to do it for all clusters within our dataframe:

In [ ]:
#Calculating the daydeltas for each cluster:
#For each unique keyword
for i in df['keyword'].unique():
    #Get just the dataframe with that keyword
    sub_df = get_subdf(df, i)
    #Create the daydelta column for sub_df
    sub_df = daydelta(sub_df, 'made_on')
    #For each row in this sub_df
    for index, row in sub_df.iterrows():
        #Assign the daydelta value to the original dataframe
        df.loc[index, 'daydelta'] = sub_df.loc[index, 'daydelta']

In [ ]:
df.info()

There should be 1 NaT value for each unique keyword (of which we have 278), therefore there are (1210-278) = 932 non-null rows in the day delta column.

Converting it to a proper Pandas Timedelta column:

In [ ]:
#Converting daydelta column to timedelta 
df['daydelta'] = pd.to_timedelta(df['daydelta'])

And making a daydelta_int column that is the daydelta as an int

In [ ]:
#Extract the number of days in the timedelta
df['daydelta_int'] = df['daydelta'].dt.days

In [ ]:
df.head()

Now to classify them into recurring/non-recurring.



After a bit of experimentation with z-scores etc, I came up with a fairly simple rule for classifying recurring payments:

If the 3 most recent payments are within +-20% of their mean, it is recurring, otherwise it's non-recurring.

Let's see which payments pass this rule:

In [ ]:
def check_recurring(df, tolerance):
    #Function that checks if the last 3 daydeltas are within given tolerance
    #get 3 last daydeltas
    recent_trans = df['daydelta_int'][-3:]
    #calculate the mean of these daydeltas
    avg_amount = recent_trans.mean()
    #Define upper and lower bound given tolerance
    lower_bound = avg_amount*(1-tolerance)
    upper_bound = avg_amount*(1+tolerance)
    #check if all daydeltas are between lower and upper bound
    output = all(lower_bound <= x <= upper_bound for x in recent_trans)
    return output

#Printing keywords that pass our rule
#For each unique keyword
for keyword in df['keyword'].unique():
    #Get just the dataframe with that keyword
    sub_df = get_subdf(df, keyword)
    #If it passses our test with 20% tolerance
    if check_recurring(sub_df, 0.20) == True:
        #Print the keyword
        print(keyword)

These look like pretty sensible suggestions for recurring payments

Now to mark the entire df as True or False for recurring

In [ ]:
#For each unique keyword
for keyword in df['keyword'].unique():
    #Get the dataframe with that keyword
    sub_df = get_subdf(df, keyword)
    #If it passses our test with 20% tolerance
    if check_recurring(sub_df, 0.2) == True:
        #For the indexes within that sub_df, assign True to the 'recurring' column in the original dataframe
        df.loc[sub_df.index, 'recurring'] = True
#Assign false to all other rows
df['recurring'].fillna(False, inplace = True)

We now have our full dataframe with all transactions marked as recurring or not:

In [ ]:
df.head()

## Finding Entity Name

To make a nice looking entity name, I will just use any words which appear in all occurences of the cleaned description. This is good because it should pick out the company name, but avoid dates, payment reference numbers etc.

In [ ]:
def find_entity_name(data):
    #Param data: should be a column of a data frame containing the cleaned descriptions
    #Returns a string of words which appear in all columns
    common_words = ''
    #Turn column of strings into list of strings
    strings = list(data)
    #For each word in the first description
    for word in strings[0].split():
        #If that word is in all descriptions
        if all(word in desc for desc in strings):
            #Concat that word and a space to the common_words string
            common_words += word + ' '
    #Remove final space
    common_words = common_words.strip()
    return common_words

#Find the entity name for the dropbox recurring payments
find_entity_name(get_subdf(df, 'dropbox debit')['clean_desc'])

Now to find the other needed features - income, period_days, and typical_amount_cents.

`income`: Get the first value of the 'income' column in each sub_df. Since we categorised by credit/debit at the clustering stage, all values for income in each sub_df will be the same.

`period_days`: Average the last 3 daydeltas

`typical_amount_cents`: Average all payment amounts

In [ ]:
def get_recurring_keywords(df):
    #Small function to get a list of keywords we have defined as recurring
    keywords = df[df['recurring'] == True]['keyword'].unique()
    return list(keywords)

def generate_output(df):
    #Get the recurring keywords
    recurring_keywords = get_recurring_keywords(df)
    #Create empty list
    output = []
    #For each recurring keyword
    for keyword in recurring_keywords:
        #Create blank dictionary
        info_dict = {}
        #Get the dataframe with that keyword
        sub_df = get_subdf(df, keyword)
        #Assign features to the dictionary
        info_dict['entity_name'] = find_entity_name(sub_df['clean_desc'])
        info_dict['income'] = sub_df['income'].iloc[0]
        info_dict['period_days'] = sub_df['daydelta_int'][-3:].mean().round(0)
        info_dict['typical_amount_cents'] = sub_df['amount_cents'].mean().round(0)
        #Append the dictionary to the output list
        output.append(info_dict)
    return output

In [ ]:
generate_output(df)

## Returning summary of recurring transactions

The below combines all of the above code into one function

In [ ]:
#RUN THIS TO GET FINAL OUTPUT

#UTILITY FUNCTIONS

def get_subdf(df, keyword):
    #Small function to find just the rows containing each keyword, ordered by date
    #Param df: dataframe to be searched
    #Param keyword: keyword to be looked for
    sub_df = df[df['keyword'] == keyword]
    #Sort values by date
    sub_df = sub_df.sort_values('made_on')
    return sub_df


#MAIN FUNCTIONS

def clean_dataframe(df):
    #Convert made_on column to datetime
    df['made_on'] = pd.to_datetime(df['made_on'])
    #drop first two columns of df
    df = df.drop(df.columns[:2], axis = 1)
    #create credit/debit type column
    df['type'] = df['income'].map(lambda x: 'credit' if x == True else 'debit')
    return df



def clean_description(df):
    #Big Regex pattern to remove undesirable parts of the description
    # \b begins and ends the pattern
    # | indicates OR
    # \d{2} - 2-digit numbers - commonly used for day in date
    # JAN-DEC - commonly used for month in date
    # ON|BCC|CLP|ATM|CH|FT|BGC - common abbreviations that indicate the type of transaction

    date_pattern = r'\b(\d{2}|JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC|ON|BCC|CLP|ATM|CH|FT|BGC|DDR|\')\b'

    df['clean_desc'] = df['original_description'].str.replace(date_pattern, '')

    #replace '*', '.', or ',' with a space
    df['clean_desc'] = df['clean_desc'].str.replace(r'\*|\.|\,',' ', regex = True)

    #convert to lower case
    df['clean_desc'] = df['clean_desc'].str.lower()
    
    #create keyword column containing first word of cleaned description and credit/debit
    df['keyword'] = df['clean_desc'].map(lambda x: x.split(' ', 1)[0]) + ' ' + df['type']
    
    return df



def daydelta(df, date_col):
    #Param df: dataframe containing the date column
    #Param date_col: string of the name of the date column to be delta'd
    #Returns dataframe with daydelta column appended
    #Sort dataframe by date column
    process_df = df.sort_values(date_col)
    #Make new column that is the date_col minus the date_column in the previous row
    process_df['daydelta'] = process_df[date_col] - process_df[date_col].shift()
    return process_df

def calculate_daydelta(df):
    for i in df['keyword'].unique():
        sub_df = get_subdf(df, i)
        sub_df = daydelta(sub_df, 'made_on')
        for index, row in sub_df.iterrows():
            df.loc[index, 'daydelta'] = sub_df.loc[index, 'daydelta']
    
    df['daydelta'] = pd.to_timedelta(df['daydelta'])
    df['daydelta_int'] = df['daydelta'].dt.days
    return df


def is_recurring(df, tolerance):
    recent_trans = df['daydelta_int'][-3:]
    avg_amount = recent_trans.mean()
    lower_bound = avg_amount*(1-tolerance)
    upper_bound = avg_amount*(1+tolerance)
    output = all(lower_bound <= x <= upper_bound for x in recent_trans)
    return output

def calculate_recurring(df):
    for i in df['keyword'].unique():
        sub_df = get_subdf(df, i)
        if is_recurring(sub_df, 0.2) == True:
            df.loc[sub_df.index, 'recurring'] = True
    df['recurring'].fillna(False, inplace = True)
    return df


def find_entity_name(data):
    #Param data: should be a column of a data frame containing the cleaned descriptions
    #Returns a string of words which appear in all columns
    common_words = ''
    #Turn column of strings into list of strings
    strings = list(data)
    #For each word in the first description
    for word in strings[0].split():
        #If that word is in all descriptions
        if all(word in desc for desc in strings):
            #Concat that word and a space to the common_words string
            common_words += word + ' '
    #Remove final space
    common_words = common_words.strip()
    return common_words

def get_recurring_keywords(df):
    #Small function to get a list of keywords we have defined as recurring
    keywords = df[df['recurring'] == True]['keyword'].unique()
    return list(keywords)

def generate_output(df):
    #Get the recurring keywords
    recurring_keywords = get_recurring_keywords(df)
    #Create empty list
    output = []
    #For each recurring keyword
    for keyword in recurring_keywords:
        #Create blank dictionary
        info_dict = {}
        #Get the dataframe with that keyword
        sub_df = get_subdf(df, keyword)
        #Assign features to the dictionary
        info_dict['entity_name'] = find_entity_name(sub_df['clean_desc'])
        info_dict['income'] = sub_df['income'].iloc[0]
        info_dict['period_days'] = sub_df['daydelta_int'][-3:].mean().round(0)
        info_dict['typical_amount_cents'] = sub_df['amount_cents'].mean().round(0)
        #Append the dictionary to the output list
        output.append(info_dict)
    return output


#DRIVER FUNCTION
    
def get_recurring_transactions(df):
    df = clean_dataframe(df)
    df = clean_description(df)
    df = calculate_daydelta(df)
    df = calculate_recurring(df)
    output = generate_output(df)
    return output

transactions = pd.read_csv('example_data.csv')
recurring = get_recurring_transactions(transactions)
recurring